In [33]:
import sys
sys.path.append('/home/peilun/ml4uc_experiment')

# from model.hand_made import GNNPolicy
from model.gcn import GNNPolicy
import os
import torch
from utilities import *
import time
from gurobipy import GRB
import gurobipy as gp
# from rich.progress import Progress
from rich.progress import Progress, BarColumn, SpinnerColumn, TimeRemainingColumn, TimeElapsedColumn
from concurrent.futures import ThreadPoolExecutor
torch.set_printoptions(edgeitems=1000000, threshold=1000000)
torch.set_printoptions(precision=4, sci_mode=False)

In [34]:
instance = '10_std'
N = int(instance.split('_')[0])
T = 24
TLE = 360
GAP = 1e-3
k1=0.999
lsr=1

In [35]:
data_path_1bin = f'../datasets/{instance}/1bin/test'
files_1bin = os.listdir(data_path_1bin)
data_path_3bin = f'../datasets/{instance}/3bin/test'
files_3bin = os.listdir(data_path_3bin)
lp_files_1bin = [f for f in files_1bin if f.endswith('.lp')][::60]
lp_files_3bin = [f for f in files_3bin if f.endswith('.lp')][::60]

lp_files_1bin = [os.path.join(data_path_1bin, f) for f in lp_files_1bin]
lp_files_3bin = [os.path.join(data_path_3bin, f) for f in lp_files_3bin]


In [36]:
model_save_path_1bin = '../model_save/1bin_10_pretrain_routine/model_best1.pth'
model_save_path_3bin = '../model_save/3bin_10_pretrain_routine/model_best.pth'
model_1bin = GNNPolicy()
state_1bin_dict = torch.load(model_save_path_1bin)
model_1bin.load_state_dict(state_1bin_dict)

model_3bin = GNNPolicy()
state_3bin_dict = torch.load(model_save_path_3bin)
model_3bin.load_state_dict(state_3bin_dict)

<All keys matched successfully>

In [37]:
print(lp_files_1bin)

['../datasets/10_std/1bin/test/7_0.lp', '../datasets/10_std/1bin/test/67_5.lp', '../datasets/10_std/1bin/test/127_0.lp', '../datasets/10_std/1bin/test/187_8.lp', '../datasets/10_std/1bin/test/247_9.lp', '../datasets/10_std/1bin/test/307_8.lp']


In [38]:
def ps_accelerate(file,Predict_model):    
    k0=1-k1
    with gp.Env(empty=True) as env:
        env.setParam('OutputFlag', 0)
        env.setParam('MIPGap', GAP)
        env.start()
        m = gp.read(file,env)
        A_indices, A_values, v_nodes, c_nodes, b_vars=get_fea(m)
        
        sol = Predict_model(c_nodes, A_indices, A_values, v_nodes).sigmoid()
        return sol[b_vars==1]
        print(sol[b_vars==1])
        instance_variabels = m.getVars()
        alphas = []
        for i,v in enumerate(instance_variabels):
            if b_vars[i]==1:
                if sol[i]>=k1 or sol[i]<=k0:
                    x_star = 1 if sol[i]>=k1 else 0
                    tmp_var = m.addVar(vtype=GRB.CONTINUOUS,name=f'alpha{v.VarName}')
                    alphas.append(tmp_var)
                    m.addConstr(tmp_var >= v - x_star, name=f'alpha_up_{v.VarName}')
                    m.addConstr(tmp_var >= x_star - v, name=f'alpha_dowm_{v.VarName}')
        all_tmp = 0
        for tmp in alphas:
            all_tmp += tmp
        m.addConstr(all_tmp <= lsr, name="sum_alpha")
        m.update()
        m.Params.Threads = 1
        m.Params.TimeLimit = TLE
        begin = time.time()
        m.optimize()
        if m.Status==2 or m.Status ==9:
            return m.objVal,time.time()-begin,len(alphas)/b_vars.sum()
        else:
            return -1,time.time()-begin,len(alphas)/b_vars.sum()


In [39]:
sol_3bin = ps_accelerate(lp_files_3bin[0],model_3bin)

In [40]:
sol_1bin = ps_accelerate(lp_files_1bin[0],model_1bin)

In [53]:
loss_fn = torch.nn.MSELoss()

In [42]:
print(loss_fn(sol_3bin,sol_1bin))

tensor(1.9964, grad_fn=<BinaryCrossEntropyBackward0>)


In [56]:
train_path_1bin = f'../datasets/{instance}/1bin/valid'
files_1bin = os.listdir(train_path_1bin)
train_path_3bin = f'../datasets/{instance}/3bin/valid'
files_3bin = os.listdir(train_path_3bin)
pt_files_1bin = [f for f in files_1bin if f.endswith('.pt')]
pt_files_3bin = [f for f in files_3bin if f.endswith('.pt')]


pt_files_1bin = [os.path.join(train_path_1bin, f) for f in pt_files_1bin]
pt_files_3bin = [os.path.join(train_path_3bin, f) for f in pt_files_3bin]


In [57]:
for d1,d3 in zip(pt_files_1bin, pt_files_3bin):
    sol1 = torch.tensor(torch.load(d1)['weighted_sol'])
    sol3 = torch.tensor(torch.load(d3)['weighted_sol'])
    print(loss_fn(sol1,sol3))
    

tensor(0., dtype=torch.float64)
tensor(0., dtype=torch.float64)
tensor(0., dtype=torch.float64)
tensor(0., dtype=torch.float64)
tensor(0., dtype=torch.float64)
tensor(0., dtype=torch.float64)
tensor(0., dtype=torch.float64)
tensor(0., dtype=torch.float64)
tensor(0., dtype=torch.float64)
tensor(0., dtype=torch.float64)
tensor(0., dtype=torch.float64)
tensor(0., dtype=torch.float64)
tensor(0., dtype=torch.float64)
tensor(0., dtype=torch.float64)
tensor(0., dtype=torch.float64)
tensor(0., dtype=torch.float64)
tensor(0., dtype=torch.float64)
tensor(0., dtype=torch.float64)
tensor(0., dtype=torch.float64)
tensor(0., dtype=torch.float64)
tensor(0., dtype=torch.float64)
tensor(0., dtype=torch.float64)
tensor(0., dtype=torch.float64)
tensor(0., dtype=torch.float64)
tensor(0., dtype=torch.float64)
tensor(0., dtype=torch.float64)
tensor(0., dtype=torch.float64)
tensor(0., dtype=torch.float64)
tensor(0., dtype=torch.float64)
tensor(0., dtype=torch.float64)
tensor(0., dtype=torch.float64)
tensor(0

: 